In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

x_train = x_train[..., None]
x_test  = x_test[..., None]

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000).batch(128).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(128).prefetch(tf.data.AUTOTUNE)

In [5]:
model = models.Sequential([
    layers.Conv2D(32, 3, activation="relu", input_shape=(28,28,1)),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(10, activation="softmax")
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
        restore_best_weights=True
    )
]

In [8]:
model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/10
469/469 - 11s - 23ms/step - accuracy: 0.9223 - loss: 0.2593 - val_accuracy: 0.9819 - val_loss: 0.0581
Epoch 2/10
469/469 - 2s - 4ms/step - accuracy: 0.9762 - loss: 0.0796 - val_accuracy: 0.9877 - val_loss: 0.0389
Epoch 3/10
469/469 - 2s - 4ms/step - accuracy: 0.9831 - loss: 0.0575 - val_accuracy: 0.9882 - val_loss: 0.0364
Epoch 4/10
469/469 - 2s - 5ms/step - accuracy: 0.9858 - loss: 0.0463 - val_accuracy: 0.9893 - val_loss: 0.0285
Epoch 5/10
469/469 - 2s - 5ms/step - accuracy: 0.9880 - loss: 0.0391 - val_accuracy: 0.9910 - val_loss: 0.0273
Epoch 6/10
469/469 - 2s - 4ms/step - accuracy: 0.9896 - loss: 0.0332 - val_accuracy: 0.9915 - val_loss: 0.0251
Epoch 7/10
469/469 - 2s - 4ms/step - accuracy: 0.9911 - loss: 0.0282 - val_accuracy: 0.9921 - val_loss: 0.0235
Epoch 8/10
469/469 - 2s - 4ms/step - accuracy: 0.9922 - loss: 0.0243 - val_accuracy: 0.9928 - val_loss: 0.0238
Epoch 9/10
469/469 - 2s - 4ms/step - accuracy: 0.9928 - loss: 0.0235 - val_accuracy: 0.9926 - val_loss: 0.0237

In [9]:
loss, acc = model.evaluate(test_ds, verbose=0)
print("Test Accuracy:", acc)

Test Accuracy: 0.9921000003814697


In [10]:
pred = model.predict(x_test[:1])
print("Predicted digit:", np.argmax(pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
Predicted digit: 7
